In [1]:
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import os
import re
from collections import defaultdict

import phylogeny_fish as phylo

Checking species ID of prototypes of ancestor models

In [3]:
model_level = 2

descendants = None
if (model_level == 0):
    descendants = defaultdict(list)
    for key, val in phylo.species_to_ances_level0.items():
        descendants[val].append(key)
elif (model_level == 1):
    descendants = defaultdict(list)
    for key, val in phylo.species_to_ances_level1.items():
        descendants[val].append(key)
elif (model_level == 2):
    descendants = defaultdict(list)
    for key, val in phylo.species_to_ances_level2.items():
        descendants[val].append(key)

# load_model_path = os.path.join(load_model_dir, load_model_name)
# load_model_path = r'/home/harishbabu/projects/ProtoPNet/saved_models/vgg19/046-fish-pad1-lvl2-256-vgg19-25ppc-20ep/10_5push0.9977.pth'
load_model_path = r'/home/harishbabu/projects/ProtoPNet/saved_models/vgg19/043-fish-pad1-lvl2-256-vgg19-10ppc-20ep/10_19push0.9977.pth'
load_model_name = os.path.basename(load_model_path)
load_img_dir = os.path.dirname(load_model_path)
epoch_number_str = re.search(r'\d+', load_model_name).group(0)
prototype_info = np.load(os.path.join(load_img_dir, 'img', 'epoch-'+epoch_number_str, 'bb'+epoch_number_str+'.npy'))
prototype_img_identity = prototype_info[:, 5]
prototype_base_img_identity = None
if prototype_info.shape[1] > 6:
    prototype_base_img_identity = prototype_info[:, 6]

descendants_used = defaultdict(set)
for img_id, base_img_id in zip(prototype_img_identity, prototype_base_img_identity):
    descendants_used[img_id].add(base_img_id)

for img_id in descendants:
    print(descendants[img_id], descendants_used[img_id])


[0] {0}
[1, 2] {1}
[3] {3}
[4] {4}
[5, 6] {5, 6}
[7, 8, 9, 10, 11, 12, 13, 14] {9, 13, 7}
[15, 16] {16, 15}
[17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 37] {37, 19, 20, 21, 27, 30}
[32, 33, 34, 35, 36] {32, 33, 35}


In [5]:
import pandas as pd
import numpy as np
import os
import cv2
from multiprocessing import Lock, Pool
import time

n_threads = 4
IMG_DIR = r'/home/harishbabu/data/CUB_190_split/official/CUB_200_2011/train_bb_crop'
SAVE_PATH = r'/home/harishbabu/data/CUB_190_split/official/CUB_200_2011/train_bb_crop_224'
folders = [os.path.join(IMG_DIR, folder) for folder in next(os.walk(IMG_DIR))[1]]
target_folders = [os.path.join(SAVE_PATH, folder) for folder in next(os.walk(IMG_DIR))[1]]

In [6]:
def split_for_threads(folders):
    print(len(folders))
    n = len(folders) // n_threads
    folders = [folders[i * n:(i + 1) * n] for i in range((len(folders) + n - 1) // n )]
    while len(folders) > n_threads:
        folders[-2] += folders[-1]
        folders = folders[:-1]
    print(len(folders[-1]))
    print(len(folders))

    return folders

folders = split_for_threads(folders)
target_folders = split_for_threads(target_folders)

190
49
4
190
49
4


In [7]:
images_count = 0
for folder in folders[0]:
    images_count += len(os.listdir(folder))
print(images_count)

images_count = 0
for folder in folders[1]:
    images_count += len(os.listdir(folder))
print(images_count)

images_count = 0
for folder in folders[2]:
    images_count += len(os.listdir(folder))
print(images_count)

images_count = 0
for folder in folders[3]:
    images_count += len(os.listdir(folder))
print(images_count)

1408
1410
1409
1468
